Project Name: Electric Car prediction

Author: A. Nikfarjam

Discription: In this project I am using KNeighborsRegressor Model and SciKit Learn to predict the price of electric cars.
The Data set comes from:https://catalog.data.gov/dataset/electric-vehicle-population-data


In [1]:
#data prep and importing libraries
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import pandas as pd 

# Load the data
data = pd.read_csv("Electric_Vehicle_Population_Data.csv")

# Weed out the zeros
data = data[['Model Year','Make', 'Model', 'Electric Range', 'Base MSRP']]
data.head()


,Model Year,Make,Model,Electric Range,Base MSRP
0,2016,BMW,X5,14.0,0.0
1,2018,TESLA,MODEL 3,215.0,0.0
2,2014,CHEVROLET,VOLT,38.0,0.0
3,2021,CHRYSLER,PACIFICA,32.0,0.0
4,2018,TESLA,MODEL 3,215.0,0.0


In [2]:
# Fill missing values
data['Electric Range'] = data['Electric Range'].fillna(0)
data['Base MSRP'] = data['Base MSRP'].fillna(0)

1- In this part I am going to Identify the features and targey variables
2- we need to split our data frame for training and testing
3- then run the the module on out test data set

In [3]:
# Split the features and target variable
cp_df = data.copy()
X = cp_df.drop('Base MSRP', axis=1)
y = cp_df['Base MSRP']  

# Dummy encoding for categorical variables on combined dataset
X_encoded = pd.get_dummies(X, columns=['Make', 'Model'])
X_encoded.head()


,Model Year,Electric Range,Make_ALFA ROMEO,Make_AUDI,Make_AZURE DYNAMICS,Make_BENTLEY,Make_BMW,Make_CADILLAC,Make_CHEVROLET,Make_CHRYSLER,...,Model_V60,Model_VOLT,Model_WHEEGO,Model_WRANGLER,Model_X3,Model_X5,Model_XC40,Model_XC60,Model_XC90,Model_XM
0,2016,14.0,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,2018,215.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2014,38.0,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
3,2021,32.0,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,2018,215.0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Instantiate the KNeighborsRegressor
reg = KNeighborsRegressor(n_neighbors=2)

# Fit the model
reg.fit(X_train, y_train)

# Make predictions on test data
y_pred = reg.predict(X_test)

#lets see out perdictions
print("This is the 10 examples of perdictions made during trainings",[x for x in y_pred if x > 0][:10])
print("This is the 10 examples of perdictions made during testing",[x for x in y_test if x > 0][:10])

This is the 10 examples of perdictions made during trainings [34995.0, 45600.0, 31950.0, 39995.0, 69900.0, 31950.0, 64950.0, 69900.0, 69900.0, 69900.0]
This is the 10 examples of perdictions made during testing [34995.0, 45600.0, 31950.0, 39995.0, 69900.0, 31950.0, 64950.0, 69900.0, 69900.0, 69900.0]


now the perdictions are made lets orgenize the data

In [5]:
#add the predicrions column in to the data frame
X_test['estimatedPrice']=y_test
Year = []
Make = []
Perdiction = []
for car in X_test.columns[2:-1]:
    true_row = X_test[car].idxmax()
    Year.append(X_test.loc[true_row, 'Model Year'])
    Make.append(car)
    Perdiction.append(X_test.loc[true_row, 'estimatedPrice'])
print(Year[:10])
print(Make[:10])
print(Perdiction[:10])
    

[2024, 2024, 2022, 2022, 2022, 2024, 2016, 2020, 2024, 2017]
['Make_ALFA ROMEO', 'Make_AUDI', 'Make_AZURE DYNAMICS', 'Make_BENTLEY', 'Make_BMW', 'Make_CADILLAC', 'Make_CHEVROLET', 'Make_CHRYSLER', 'Make_DODGE', 'Make_FIAT']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [6]:
prd_df = pd.DataFrame({'Year': Year,
                       'Make': Make,
                       'Predicted Price$': Perdiction
                       })
prd_df[prd_df['Predicted Price$']>0]

,Year,Make,Predicted Price$
33,2019,Make_SUBARU,34995.0
42,2018,Model_530E,54950.0
43,2018,Model_740E,90700.0
67,2019,Model_COUNTRYMAN,36900.0
68,2019,Model_CROSSTREK,34995.0
69,2018,Model_CT6,75095.0
113,2012,Model_KARMA,102000.0
146,2008,Model_ROADSTER,98950.0
158,2016,Model_SOUL,31950.0
